### Tools in LangChain
Tools are utilities designed to be called by a model: their inputs are designed to be generated by models, and their outputs are designed to be passed back to models.

In [4]:
from langchain_community.tools import DuckDuckGoSearchRun

search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke('weather news india')

print(result)

15 hours ago - Weather today: A fresh Western Disturbance is set to bring rain and snowfall to the western Himalayas, along with dense fog, hailstorms and biting cold conditions across several parts of India , said the IMD 1 week ago - To highlight temperature extremes, the page also features regularly updated lists of the Top 5 Hottest Cities and Top 5 Coldest Cities in India , giving readers instant insight into heatwave and cold-wave conditions across different parts of the country. In addition to live data, the page curates the latest weather-related news from across India, including reports on heatwaves, monsoon progress, extreme rainfall, cyclones, cold spells, and advisories issued by the India Meteorological Department (IMD). 2 weeks ago - Get the latest weather forecast for India with real-time updates on today’s temperature, humidity, and conditions. Plan your day with confidence using accurate, up-to-date weather insights for cities across India. 3 days ago - In an advisory,

In [7]:
from langchain_community.tools import ShellTool

shell_tool = ShellTool()

result = shell_tool.invoke('whoami')

print(result)

Executing command:
 whoami
gaurvit\gaurv



c:\Users\gaurv\Downloads\STUDY\Generative AI\LangChain\venv\Lib\site-packages\langchain_community\tools\shell\tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


### Custom Tools

In [8]:
from langchain_community.tools import tool


# A simple tool to multiply two numbers, it is recommended to give type hints and a docstring while making a custom tool
# As it tells the LLM what the tool does and takes as input.
@tool
def multiply(a: int, b: int)->int:
    """Multiplies two integer numbers"""
    return a * b

In [9]:
result = multiply.invoke({"a" : 10, "b" : 5})

print(result)

50


In [10]:
print(multiply.name)
print(multiply.description) # This is the docstring we provided in the tool definition
print(multiply.args) # These are the parameters along with type checks given in the tool

multiply
Multiplies two integer numbers
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [11]:
# This is what the LLM sees when the tool is passed to it rather than the function
print(multiply.args_schema.model_json_schema())

{'description': 'Multiplies two integer numbers', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'multiply', 'type': 'object'}


### Structured Tools

In [12]:
from langchain_community.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Annotated

In [13]:
class MultiplyInput(BaseModel):
    a: Annotated[int, Field(..., description="The first number for multiplication")]
    b: Annotated[int, Field(..., description="The second number for multiplication")]

In [16]:
def multiply_func(a: int, b: int)->int:
    return a * b

In [17]:
multiply_tool = StructuredTool.from_function(
    func=multiply_func,
    name="multiply",
    description="Multiplies two numbers",
    args_schema=MultiplyInput
)

In [19]:
result = multiply_tool.invoke({"a" : 12, "b" : 10})

print(result)

120


In [20]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

multiply
Multiplies two numbers
{'a': {'description': 'The first number for multiplication', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number for multiplication', 'title': 'B', 'type': 'integer'}}


In [18]:
print(multiply_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number for multiplication', 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number for multiplication', 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


### BaseTool Class

In [21]:
from langchain.tools import BaseTool
from typing import Type

In [22]:
class MultiplyInput(BaseModel):
    a: Annotated[float, Field(..., description="The first number for multiplication")]
    b: Annotated[float, Field(..., description="The second number for multiplication")]

In [ ]:
class MultiplyTool(BaseTool):
    name: str = "Multiply"
    description: str = "Multiplies two numbers"

    args_schema: Type[BaseModel] = MultiplyInput

    def _run(self, a: int, b: int)->int: # The method should be named _run (mandatory)
        return a * b

In [24]:
multiply_tool = MultiplyTool()

In [25]:
result = multiply_tool.invoke({"a" : 15.25, "b" : 1.56})

print(result)

23.79


In [26]:
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args)

Multiply
Multiplies two numbers
{'a': {'description': 'The first number for multiplication', 'title': 'A', 'type': 'number'}, 'b': {'description': 'The second number for multiplication', 'title': 'B', 'type': 'number'}}


In [27]:
print(multiply_tool.args_schema.model_json_schema())

{'properties': {'a': {'description': 'The first number for multiplication', 'title': 'A', 'type': 'number'}, 'b': {'description': 'The second number for multiplication', 'title': 'B', 'type': 'number'}}, 'required': ['a', 'b'], 'title': 'MultiplyInput', 'type': 'object'}


### Toolkit in LangChain
A toolkit is just a collection (bundle) of related tools that serve a common purpose packaged together for convenience and reusability.

In [ ]:
from langchain.tools import tool

# Creating Custom Tools to be bundled as a Toolkit
@tool
def add(a: float, b: float)->float:
    """Adds two numbers"""
    return a + b

@tool
def subtract(a: float, b: float)->float:
    """Subtracts two numbers"""
    return a - b

@tool
def mutliply(a: float, b: float)->float:
    """Multiplies two numbers"""
    return a * b

@tool
def divide(a: float, b: float)->float:
    """Divides two numbers"""
    return a / b

In [ ]:
# Custom Toolkit for various Math Operations
class MathToolkit:
    def get_tools(self):
        return [add, subtract, mutliply, divide]

In [ ]:
# Creating an object of Custom Toolkit Class
math_toolkit = MathToolkit()

# Extracting tools from the toolkit object
math_tools = math_toolkit.get_tools()

for tool in math_tools:
    print(f"Name : {tool.name}, Description : {tool.description}")

Name : add, Description : Adds two numbers
Name : subtract, Description : Subtracts two numbers
Name : mutliply, Description : Multiplies two numbers
Name : divide, Description : Divides two numbers
